In [ ]:
import os
import cv2

# 将当前工作目录更改为指定路径
os.chdir('C:/xxx/xxx/')

# 检查当前工作目录
print("当前工作目录:", os.getcwd())
#创建yolo10内核
!conda create -n yolov10 python=3.9


In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from ultralytics import YOLOv10
from datetime import datetime
import time

# 加载模型
model = YOLOv10('./mark2.pt')

# 定义像素到微米的转换比率（46像素 = 100微米）
pixel_to_micron = 100 / 40

# 打开摄像头，使用设备索引0（通常是默认摄像头）
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# 检查摄像头是否成功打开
if not cap.isOpened():
    print("无法打开摄像头")
    exit()

# 设置摄像头分辨率（可选）
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# 获取摄像头信息
fps = int(cap.get(cv2.CAP_PROP_FPS))
if fps == 0:  # 如果无法获取fps，设置默认值
    fps = 30
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"摄像头分辨率: {width}x{height}, FPS: {fps}")

# 获得时间戳
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# 初始化视频写入器
output_path = f"./camera_{current_time}.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 存储每秒的直径、间距和卫星液滴数量数据
diameters = []
diameters_per_second = []
distances_per_second = []
satellite_counts_per_second = []

# 计时变量
frame_count = 0
second_diameters = []
second_centers = []
second_distance = []
satellite_count = 0
start_time = time.time()  # 记录开始时间

def mean_ignore_zeros(values):
    non_zero_values = [v for v in values if v != 0]
    return np.mean(non_zero_values) if non_zero_values else 0

def std_ignore_zeros(values):
    non_zero_values = [v for v in values if v != 0]
    return np.std(non_zero_values) if non_zero_values else 0

def append_if_valid(value, target_list):
    if not np.isnan(value):
        target_list.append(value)
    else:
        print(f"Skipped NaN value: {value}")

print("开始摄像头检测，按 'q' 键退出...")

# 遍历摄像头的每一帧
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("无法读取摄像头帧")
        break

    # 检测
    results = model(frame, conf=0.25)
    detections = results[0].boxes

    # 存储当前帧的液滴直径和中心坐标
    frame_diameters = []
    frame_centers = []

    for box in detections:
        class_id = int(box.cls.cpu())  # 获取物体的类别
        x_center, y_center, width_box, height_box = box.xywh[0].cpu()  # 获取物体的中心坐标和长宽

        # 只处理 class_id 为 0 或 1 的物体
        if class_id == 0 or class_id == 1:  # 液滴标签或卫星液滴标签
            # 计算长圆形的面积并得到等效圆直径
            L = width_box  # 矩形框的长边
            W = height_box  # 矩形框的短边
            area_obround = (L - W) * W + (np.pi * W ** 2) / 4  # 计算长圆形的面积
            equivalent_diameter = 2 * np.sqrt(area_obround / np.pi)  # 计算等效圆形直径
            diameter_microns = equivalent_diameter * pixel_to_micron  # 转换为微米

            if class_id == 0:  # 液滴标签
                append_if_valid(diameter_microns, frame_diameters)
                append_if_valid(value=diameter_microns, target_list=diameters)
                frame_centers.append((x_center, y_center))
            elif class_id == 1:  # 卫星液滴标签
                satellite_count += 1  # 卫星液滴数量统计

            # 画框
            top_left = (int(x_center - width_box / 2), int(y_center - height_box / 2))
            bottom_right = (int(x_center + width_box / 2), int(y_center + height_box / 2))
            cv2.rectangle(frame, top_left, bottom_right, (255, 0, 0), 2)
            cv2.circle(frame, (int(x_center), int(y_center)), 5, (0, 0, 255), -1)
            label = f"D: {diameter_microns:.2f} μm"
            cv2.putText(frame, label, (top_left[0], top_left[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # 自动选择排序方向
    if len(frame_centers) > 1:
        x_range = max(c[0] for c in frame_centers) - min(c[0] for c in frame_centers)
        y_range = max(c[1] for c in frame_centers) - min(c[1] for c in frame_centers)
        if x_range >= y_range:
            frame_centers.sort(key=lambda c: c[0])  # 按 x 坐标排序
        else:
            frame_centers.sort(key=lambda c: c[1])  # 按 y 坐标排序

    # 计算相邻液滴间距
    frame_distances = []
    for i in range(len(frame_centers) - 1):
        x1, y1 = frame_centers[i]
        x2, y2 = frame_centers[i + 1]
        distance = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * pixel_to_micron
        frame_distances.append(distance)

    # 添加当前帧数据到一秒的累计数据中
    second_diameters.extend(frame_diameters)
    second_distance.extend(frame_distances)
    frame_count += 1

    # 检查是否过了一秒
    elapsed_time = time.time() - start_time
    if elapsed_time >= 1.0:  # 每秒处理一次
        # 计算每秒平均粒径
        avg_diameter = mean_ignore_zeros(second_diameters)
        diameters_per_second.append(avg_diameter)

        # 计算平均间距
        avg_distance = mean_ignore_zeros(second_distance)
        distances_per_second.append(avg_distance)

        # 添加卫星液滴数量
        satellite_counts_per_second.append(satellite_count)

        # 重置一秒的累计数据
        second_diameters.clear()
        second_centers.clear()
        frame_count = 0
        satellite_count = 0
        start_time = time.time()  # 重置计时

    # 写入视频帧
    out.write(frame)
    
    # 显示实时画面（可选）
    cv2.imshow('Camera Detection', frame)
    
    # 按 'q' 键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放资源
cap.release()
out.release()
cv2.destroyAllWindows()

# 保存直径数据到 txt 文件
txt_path = f"./diameters_{current_time}.txt"
with open(txt_path, 'w') as f:
    for d in diameters:
        f.write(f"{d:.2f}\n")

print(f"液滴直径数据已保存至: {txt_path}")

# 计算平均值和变异系数
if diameters:  # 确保有数据
    # 转换为 NumPy 数组
    diameters_np = np.array(diameters)

    # 计算非零值的均值和标准差
    mean_diameter = mean_ignore_zeros(diameters_np)
    std_diameter = std_ignore_zeros(diameters_np)

    # 计算变异系数
    coefficient_of_variation = std_diameter / mean_diameter if mean_diameter > 0 else 0

    print(f"平均直径: {mean_diameter:.2f} μm")
    print(f"变异系数: {coefficient_of_variation:.4f}")

    # 只有当有足够数据时才绘制图表
    if len(diameters_per_second) > 1:
        # 计算导数
        diameter_derivative = np.diff(diameters_per_second)
        distance_derivative = np.diff(distances_per_second)
        satellite_count_derivative = np.diff(satellite_counts_per_second)
        time_seconds = range(1, len(diameters_per_second))  # 从1秒开始，因为导数减少了一个数据点

        # 绘制导数随时间的分布图
        plt.figure(figsize=(12, 8))
        plt.plot(time_seconds, diameter_derivative, label="diameters", marker='o', color='b', markersize=3)
        plt.plot(time_seconds, distance_derivative, label="distances", marker='x', color='g', markersize=3)
        plt.plot(time_seconds, satellite_count_derivative, label="satellite", marker='s', color='r', markersize=3)

        plt.xlabel('Time (s)')
        plt.ylabel('Derivative Values')
        plt.title('Derivatives of Average Diameter, Average Distance, and Satellite Droplet Count Over Time')
        plt.legend()
        plt.grid(True)
        plt.show()

        # 绘制液滴直径随时间的分布图
        save_path = f"./diameters_over_time_{current_time}.png"

        plt.figure(figsize=(12, 8))
        plt.plot(range(len(diameters_per_second)), diameters_per_second, marker='o', color='b', markersize=3)

        plt.xlabel('Time (s)')
        plt.ylabel('diameter(μm)')
        plt.title('diameter to time')
        plt.grid(color='gray', linestyle='--', linewidth=0.5)

        # 保存图像到指定路径
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        print(f"Plot saved at: {save_path}")
else:
    print("没有检测到液滴数据")
